In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Cesta k základnímu modelu a vstupním datům
base_model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"  
csv_file = "CSV-Hashed.csv"

# Načtení modelu
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model.eval()

# Načtení tokenizeru
tokenizer = AutoTokenizer.from_pretrained(
    base_model_path,
    trust_remote_code=True
)

# Načtení a zpracování CSV
df = pd.read_csv(csv_file)
csv_text = df.to_csv(index=False)  # převedeno na text

# Vytvoření promptu
prompt = f"I will give you CSV and your job is to find frequent itemsets of maxlenght 5 with support 0.75. Do not use any code, it is your job to do!\n{csv_text}\n\n"

# Tokenizace vstupu
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generování výstupu
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=5000, # Zvolena vysoká hodnota pro možnost generování rozsáhlých odpovědí. Vyšší hodnota způsobovala zamrznutí modelu (nízká VRAM)
        temperature=1  # nebo 0, nebo 0.5
    )
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# === Uložení výstupu ===
with open("model_output.txt", "w", encoding="utf-8") as f:
    f.write(decoded_output)

print("✅ Výstup byl uložen do 'model_output.txt'")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


✅ Výstup byl uložen do 'model_output.txt'
